## 은행 마케팅
* 은행의 데이터를 모델링하여 고객의 정기예금 가입 여부를 예측하고자 한다
* train, test셋 7:3
* 성능이 제일 좋은 모델을 찾으시고

* 요약
    * 데이터는 포르투갈 은행 기관의 직접 마케팅 전화통화와 관련이 있다
    * 분류 목표는 고객이
        * 정기예금에 가입할지 여부를 예측하는 것이다
* 컬럼
    * age : 나이(숫자)
    * duration : 지속 시간, 마지막 접촉 지속 시간(초)(숫자). 중요
        * 참고 사항 : 이 속성은 통화 대상에 큰 영향을 미칩니다 (예 : duration=0이면, y=0 => 가입하지 않음)
    * campaign(마케팅 전화통화) : 이 캠페인 동안 클라이언트에 대해 가입(컨택) 제의를 받은 수(숫자, 마지막 컨택 포함)
    * Pdays : 이전 통화에서 클라이언트가 마지막으로 연락 된 후 경과한 일 수
        * (숫자, 999는 클라이언트가 이전에 연락되지 않았음을 의미
    * previous : 현재 통화 이전에 이 클라이언트에 대해 수행된 가입(컨택) 제의를 받은 수(숫자)
    * y : 고객이 정기 예금에 가입했습니까? (예 : 1, 아니오 : 0)

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/bank_cleaning.csv")
df

,age,duration,campaign,pdays,previous,y
0,56,261,1,999,0,0
1,57,149,1,999,0,0
2,37,226,1,999,0,0
3,40,151,1,999,0,0
4,56,307,1,999,0,0
...,...,...,...,...,...,...
41183,73,334,1,999,0,1
41184,46,383,1,999,0,0
41185,56,189,2,999,0,0
41186,44,442,1,999,0,1


In [3]:
df.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'y'], dtype='object')

In [4]:
features = [ 'age', 'duration', 'campaign', 'pdays', 'previous' ]
label = 'y'

X = df[features]
y = df[label]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3 )

print("총 개수 : ", X.shape, y.shape)
print("train 개수 : ", X_train.shape, y_train.shape)
print("test 개수 : ", X_test.shape, y_test.shape)

총 개수 :  (41188, 5) (41188,)
train 개수 :  (28831, 5) (28831,)
test 개수 :  (12357, 5) (12357,)


In [6]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit( X_train, y_train )
kn.score( X_test, y_test )

0.8997329448895363

In [ ]:
# 학습 시간이 꽤 걸린다. 15분째 기다리는데 안된다... 흠....
import sklearn.svm as svm

svm_linear = svm.SVC(kernel="linear")
svm_linear.fit(X_train, y_train)
svm_linear.score(X_test, y_test)

In [10]:
# 약 1분정도 소요된다.

svm_rbf = svm.SVC(kernel="rbf")
svm_rbf.fit( X_train, y_train )
svm_rbf.score( X_test, y_test )

0.9081492271586955

In [11]:
from sklearn.tree import DecisionTreeClassifier

# 모델 얻어오기
dt = DecisionTreeClassifier()

# 학습을 통한 모델 생성
dt.fit( X_train, y_train )
dt.score( X_test, y_test )

0.863720967872461

In [12]:
svm_rvot = svm.SVC(kernel="rbf", probability=True)
svm_rvot.fit( X_train, y_train )

kn = KNeighborsClassifier()
kn.fit( X_train, y_train )

dt = DecisionTreeClassifier()
dt.fit( X_train, y_train )

DecisionTreeClassifier()

In [13]:
from sklearn.ensemble import VotingClassifier

vo = VotingClassifier( estimators=[ ("svc", svm_rvot), ("knn", kn), ("DecisionTree", dt) ], voting="soft" )
vo.fit( X_train, y_train )

print("svm : ", svm_rvot.score( X_test, y_test ))
print("kn : ", kn.score( X_test, y_test ))
print("dt : ", dt.score( X_test, y_test ))
print("vo : ", vo.score( X_test, y_test ))

svm :  0.9081492271586955
kn :  0.8997329448895363
dt :  0.8646111515740066
vo :  0.90102775754633


In [14]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit( X_train, y_train )
rfc.score( X_test, y_test )

0.8930161042324188

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gbc.fit( X_train, y_train )
gbc.score( X_test, y_test )

0.9099295945617868

## 예측
* age : 나이(숫자)
* duration : 지속 시간, 마지막 접촉 지속 시간(초)(숫자). 중요
    * 참고 사항 : 이 속성은 통화 대상에 큰 영향을 미칩니다 (예 : duration=0이면, y=0 => 가입하지 않음)
* campaign(마케팅 전화통화) : 이 캠페인 동안 클라이언트에 대해 가입(컨택) 제의를 받은 수(숫자, 마지막 컨택 포함)
* Pdays : 이전 통화에서 클라이언트가 마지막으로 연락 된 후 경과한 일 수
    * (숫자, 999는 클라이언트가 이전에 연락되지 않았음을 의미
* previous : 현재 통화 이전에 이 클라이언트에 대해 수행된 가입(컨택) 제의를 받은 수(숫자)
* y : 고객이 정기 예금에 가입했습니까? (예 : 1, 아니오 : 0)

In [16]:
df.head(2)

,age,duration,campaign,pdays,previous,y
0,56,261,1,999,0,0
1,57,149,1,999,0,0


In [17]:
result = gbc.predict( [ [ 56, 261, 1, 999, 0 ] ] )

if (result[0] == 0) :
    print("가입하기엔 노오력이 부족하다...")
else :
    print("쪼오끔만 더...")

가입하기엔 노오력이 부족하다...
